# Hypothesis

Buying when RSI crosses below oversold threshold selling when RSI crosses above the overbought threshold will outperform buy and hold. 

**Steps**
1. Get 2 years of TSLA daily candles
2. Add RSI Signal
3. Add trade column containing:
    * Buy if: previous rows value is blank; and RSI < 30   
    * Hold if: previous rows value is Buy or Hold; and RSI < 30
    * Sell if: previous rows value is Buy or Hold; and RSI > 70
    * Blank if: previous rows value is Sell or blank; and RSI > 70
4. Add cumulative profit column which is updated on Sell
5. Calculate profit from buy and hold vs total cumulative profit from above strategy
    
**Assumptions**
* Trades cost £10 in both directions (source IG Markets)

## Step 1: Get the candles 
When run on 5DEC21 Yahoo finance only goes back as far as 15MAR21. Downloaded manually as csv file in ~/lab/data. Data from 8OCT21 to 3DEC21. This gives us 2 years worth + some extra days for RSI calculations.

In [1]:
import pandas as pd
import datetime as dt

# Load from CSV and sort
candles = pd.read_csv('~/data/TSLA.csv')
candles = candles.sort_index()

candles

,Date,Open,High,Low,Close,Adj Close,Volume
0,2019-10-08,47.174000,48.787998,46.900002,48.009998,48.009998,43391000
1,2019-10-09,48.264000,49.459999,48.130001,48.905998,48.905998,34472000
2,2019-10-10,49.056000,49.855999,48.316002,48.948002,48.948002,31416500
3,2019-10-11,49.430000,50.216000,49.362000,49.577999,49.577999,42377000
4,2019-10-14,49.580002,51.709999,49.425999,51.391998,51.391998,51025000
...,...,...,...,...,...,...,...
540,2021-11-29,1100.989990,1142.670044,1100.189941,1136.989990,1136.989990,19464500
541,2021-11-30,1144.369995,1168.000000,1118.000000,1144.760010,1144.760010,27092000
542,2021-12-01,1160.699951,1172.839966,1090.760010,1095.000000,1095.000000,22816800
543,2021-12-02,1099.060059,1113.000000,1056.650024,1084.599976,1084.599976,24371600


## Step 2: Add the indicators
Calculate RSI:
* RSI from pandas_ta

In [2]:
import pandas_ta as ta

# Copy data first so that we dont change the candles
ta_data = candles.copy()

# Add Indicators
ta_data['RSI'] = ta_data.ta.rsi(close=ta_data['Close'])

# Remove first 14 rows used to calculate the first RSIs
ta_data = ta_data.iloc[14: , :].copy()

ta_data

,Date,Open,High,Low,Close,Adj Close,Volume,RSI
14,2019-10-28,65.508003,68.167999,64.519997,65.542000,65.542000,94351500,90.870948
15,2019-10-29,63.998001,64.860001,62.950001,63.243999,63.243999,63421500,81.470502
16,2019-10-30,62.599998,63.757999,61.993999,63.001999,63.001999,48209000,80.525771
17,2019-10-31,62.619999,63.799999,62.599998,62.984001,62.984001,25335000,80.451051
18,2019-11-01,63.264000,63.296001,61.959999,62.661999,62.661999,31919500,79.038016
...,...,...,...,...,...,...,...,...
540,2021-11-29,1100.989990,1142.670044,1100.189941,1136.989990,1136.989990,19464500,59.016974
541,2021-11-30,1144.369995,1168.000000,1118.000000,1144.760010,1144.760010,27092000,59.750833
542,2021-12-01,1160.699951,1172.839966,1090.760010,1095.000000,1095.000000,22816800,53.182964
543,2021-12-02,1099.060059,1113.000000,1056.650024,1084.599976,1084.599976,24371600,51.898928


## Step 3: Calculate the trades
* Define function to calculate trade signal. This will need previous rows trade, and current rows RSI. Previous trade will be stored as global as lambda won't have access to previous rows for this calculation. This should return:
    * BUY if: previous trade is 'NONE'; and RSI < 30
    * HOLD if: previous trade is BUY or HOLD; and RSI < 30
    * SELL if: previous trade is BUY or HOLD; and RSI > 70
    * NONE if: previous trade is SELL or 'NONE; and RSI > 70
* Use lamda to apply to dataframe

In [3]:
# Define the calc_trade function
def calc_trade(rsi):
    # Access trade global. This will hold the previous trade for each itteration.
    global trade
    
    # Change value of trade depending on previous trade and indicators
    if trade == 'NONE' and rsi < 30:
        trade = 'BUY'
    elif trade in ['BUY', 'HOLD'] and rsi < 30:
        trade = 'HOLD'
    elif trade in ['BUY', 'HOLD'] and rsi > 70:
        trade = 'SELL'
    elif trade in ['NONE', 'SELL'] and rsi > 70:
        trade = 'NONE'
            
    return trade

# Set initial value for trade and apply calc_trade to a copy of the dataframe
trade = 'NONE'
trade_data = ta_data.copy()
trade_data['trade'] = trade_data.apply(lambda row : calc_trade(row['RSI']), axis = 1)

trade_data.loc[trade_data['trade'] != 'NONE']

,Date,Open,High,Low,Close,Adj Close,Volume,RSI,trade
109,2020-03-16,93.900002,98.973999,88.433998,89.014000,89.014000,102447500,28.444760,BUY
110,2020-03-17,88.001999,94.370003,79.199997,86.040001,86.040001,119973000,27.654393,HOLD
111,2020-03-18,77.800003,80.972000,70.101997,72.244003,72.244003,118931000,24.283571,HOLD
112,2020-03-19,74.940002,90.400002,71.692001,85.528000,85.528000,150977500,32.779899,HOLD
113,2020-03-20,87.639999,95.400002,85.157997,85.505997,85.505997,141427500,32.773339,HOLD
...,...,...,...,...,...,...,...,...,...
505,2021-10-08,796.210022,796.380005,780.909973,785.489990,785.489990,16711100,62.002860,HOLD
506,2021-10-11,787.650024,801.239990,785.500000,791.940002,791.940002,14200300,64.166442,HOLD
507,2021-10-12,800.929993,812.320007,796.570007,805.719971,805.719971,22020000,68.317123,HOLD
508,2021-10-13,810.469971,815.409973,805.780029,811.080017,811.080017,14120100,69.783285,HOLD


## Step 4: Calculate cumulative profit
* Filter to trade rows only.
* Add profit to every SELL row by taking close price - previous rows close price. Previous row will be the corresponding BUY
* Sum the profits for cumulative profit

In [4]:
# Get the trade rows
trades = trade_data.loc[trade_data['trade'].isin(['BUY', 'SELL'])].copy()

# Create a shift column containing the previous rows Close value so that we can access in lambda function to calculate profit
trades['prev_close'] = trades['Close'].shift(1)

# Define method to calculate. Will return close - prev_close - broker_fees if trade_direction is SELL. 
# Otherwise will return None
def calculate_profit(trade_direction, close, prev_close, broker_fees):
    profit = None
    if trade_direction == 'SELL':
        profit = close - prev_close - broker_fees  
        
    return profit

trades['profit_inc_fees'] = trades.apply(lambda row : calculate_profit(row['trade'], row['Close'], row['prev_close'], 20), axis = 1)
trades['profit_excl_fees'] = trades.apply(lambda row : calculate_profit(row['trade'], row['Close'], row['prev_close'], 0), axis = 1)


cumulative_profit_inc_fees = trades['profit_inc_fees'].sum()
cumulative_profit_excl_fees = trades['profit_excl_fees'].sum()

cumulative_profit_excl_fees

319.489998

## Step 5: Compare cumulative profit vs buy and hold
* Compare against buy and hold, buying at first row and selling at last
* Compare against buy and hold, buying at first BUY and selling at last SELL

In [5]:
# Cumulative profit from strategy excluding fees. Buy price is taken as first buy
buy_price = trade_data['Close'].iloc[0]
profit = cumulative_profit_excl_fees
profit_pct = profit / buy_price * 100
print(f"Strategy: Buying and selling using strategy but ignoring broker fees generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

# Cumulative profit from strategy including fees. Buy price is taken as first buy
buy_price = trade_data['Close'].iloc[0]
profit = cumulative_profit_inc_fees
profit_pct = profit / buy_price * 100
print(f"Strategy: Buying and selling using strategy including broker fees generated ${profit:,.2f}. A return of {profit_pct:.2f}%")


# Profit from last row - first row
buy_price = trade_data['Close'].iloc[0]
sell_price = trade_data['Close'].iloc[-1]

profit = sell_price - buy_price
profit_pct = profit / buy_price * 100

print(f"First vs Last Row: Buying at ${buy_price:,.2f} and selling at ${sell_price:,.2f} generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

# Profit from last SELL - first BUY
buy_price = trades[trades['trade'] == 'BUY']['Close'].iloc[0]
sell_price = trades[trades['trade'] == 'SELL']['Close'].iloc[-1]

profit = sell_price - buy_price
profit_pct = profit / buy_price * 100

print(f"First Buy vs Last Sell: Buying at ${buy_price:,.2f} and selling at ${sell_price:,.2f} generated ${profit:,.2f}. A return of {profit_pct:.2f}%")

Strategy: Buying and selling using strategy but ignoring broker fees generated $319.49. A return of 487.46%
Strategy: Buying and selling using strategy including broker fees generated $259.49. A return of 395.91%
First vs Last Row: Buying at $65.54 and selling at $1,014.97 generated $949.43. A return of 1448.58%
First Buy vs Last Sell: Buying at $89.01 and selling at $818.32 generated $729.31. A return of 819.32%


## Conclusion
With the data available, buying when RSI crosses below oversold threshold selling when RSI crosses above the overbought threshold underperformed both a simple buy and hold and a RSI timed buy and hold. This was the case when ignoring broker fees and when accounting for fees. 